In [1]:
import json
from pathlib import Path
import requests
from bs4 import BeautifulSoup

import time

from tqdm.notebook import trange, tqdm

import pandas as pd
import numpy as np

In [2]:
## 공공 데이터 API 키 설정

apiKeyPath = './api-key/public-data-gov-key.json'
apiKey = ''

with open(apiKeyPath) as json_file:
    loaded = json.load(json_file)
    apiKey = loaded['key']

In [10]:
## 법정동 코드 Pandas 로 읽기
dfDistrictCodeB = pd.read_csv('./dataset/district_code_b.csv').query("폐지여부 == False")
dfDistrictCodeB['법정구코드'] = dfDistrictCodeB['법정동코드'].apply(lambda x: int(str(x)[:5]))

In [14]:
print(len(dfDistrictCodeB['법정구코드'].unique()))

277


In [20]:
## 아파트 실거래가 데이터
def getAptTradeResponse(code, ymd, key):
    apiPrefixGetAptTrade = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"
    urlGetAptTrade = "{}?type=xml&LAWD_CD={}&DEAL_YMD={}&serviceKey={}".format(
        apiPrefixGetAptTrade, 
        code,
        ymd,
        key
    )

    responseApiTrade = requests.get(urlGetAptTrade, verify = False)
    return responseApiTrade.text

# XML 파싱 후 필터링
def parseAptTradeResponse(text):
    parsed = BeautifulSoup(text, 'lxml-xml')
    rawXmlItems = parsed.findAll("item")

    colNames = [
        '법정동',
        '지역코드',
        '아파트',
        '지번',
        '년',
        '월',
        '일',
        '건축년도',
        '전용면적',
        '층',
        '거래금액'
    ]

    dfRaw = pd.DataFrame(index = [], columns = colNames)

    # XML 내 item 을 반복하며 pandas df 로 변경
    for rawXmlItem in rawXmlItems: 
        row = dict.fromkeys(colNames)

        for col in colNames:       
            try :
                row[col] = rawXmlItem.find(col).text
            except :
                row[col] = np.nan

        dfTemp = pd.DataFrame(
            [row],
            columns = colNames)

        dfRaw = dfRaw.append(dfTemp)

    # 컬럼 타입 변경 또는 추가
    dfReturn = dfRaw
    dfReturn['거래일'] = pd.to_datetime(dfRaw['년'] + '-' + dfRaw['월'] + '-' + dfRaw['일'])
    dfReturn['거래금액'] = pd.to_numeric(dfRaw['거래금액'].str.replace(',', ''))

    dfReturn['지역코드'] = pd.to_numeric(dfRaw['지역코드'])
    dfReturn['년'] = pd.to_numeric(dfRaw['년'])
    dfReturn['월'] = pd.to_numeric(dfRaw['월'])
    dfReturn['일'] = pd.to_numeric(dfRaw['일'])
    
    dfReturn['법정동'] = dfRaw['법정동'].str.strip()
    dfReturn['아파트'] = dfRaw['아파트'].str.strip()
    dfReturn['지번'] = dfRaw['지번'].str.strip()

    dfReturn['건축년도'] = pd.to_numeric(dfRaw['건축년도'])
    dfReturn['전용면적'] = pd.to_numeric(dfRaw['전용면적'])
    dfReturn['층'] = pd.to_numeric(dfRaw['층'])
    
    dfReturn['p_ymd'] = pd.Timestamp("today").strftime("%Y%m%d")
    dfReturn.index = range(len(dfReturn))

    return dfReturn

In [18]:
def crawlAptTrade(code, ymd, key):
    filePath = './dataset/apt-trade/{}/apt-trade-{}-{}.csv'.format(ymd, code, ymd)
    
    resAptTrade = getAptTradeResponse(
        code, 
        ymd,
        key)

    dfAptTrade = parseAptTradeResponse(resAptTrade)
    
    if dfAptTrade.size > 0:
        dfAptTrade.to_csv(filePath, mode='w', index=False)

In [ ]:
dealYmd = '201911'
uniqueCodes = dfDistrictCodeB['법정구코드'].unique()

print('아파트 실거래가 수집 ({})'.format(dealYmd))
for code in tqdm(uniqueCodes):
    
    print('수집중: {}'.format(code))
    
    time.sleep(0.10)
    
    crawlAptTrade(code, dealYmd, apiKey)

아파트 실거래가 수집 (201911)


수집중: 11000
수집중: 11110
수집중: 11140
수집중: 11170
수집중: 11200
수집중: 11215
수집중: 11230
수집중: 11260
수집중: 11290
수집중: 11305
수집중: 11320
수집중: 11350
수집중: 11380
수집중: 11410
수집중: 11440
수집중: 11470
수집중: 11500
수집중: 11530
수집중: 11545
수집중: 11560
수집중: 11590
수집중: 11620
수집중: 11650
수집중: 11680
수집중: 11710
수집중: 11740
수집중: 26000
수집중: 26110
수집중: 26140
수집중: 26170
수집중: 26200
수집중: 26230
수집중: 26260
